In [ ]:
# Homology calculation in SageMath

# Step 1: Define polynomial ring for coefficients
R = PolynomialRing(ZZ, [f"x{i}{j}" for i in range(3) for j in range(3)])
vars = R.gens()


In [2]:

# Step 2: Define generic 3x3 matrix A
A = matrix(R, 3, 3, vars)
print("Generic matrix A:")
show(A)


Generic matrix A:


[x00 x01 x02]
[x10 x11 x12]
[x20 x21 x22]

In [3]:

# Step 3: Define boundary map

def boundary_image(M):
    return matrix(R, 2, 2, [
        [M[1,1], M[1,2] - M[0,2] + M[0,1]],
        [M[2,1] - M[2,0] + M[1,0], M[1,1]]
    ])


In [4]:

# Step 4: Compute boundary of A
B_A = boundary_image(A)
print("\nBoundary of A:")
show(B_A)



Boundary of A:


[            x11 x01 - x02 + x12]
[x10 - x20 + x21             x11]

In [5]:
# Step 5: Build boundary matrix B
def boundary_matrix():
    basis = [matrix(R, 3, 3, {(i,j):1}) for i in range(3) for j in range(3)]
    rows = []
    for i in range(2):
        for j in range(2):
            row = [boundary_image(E)[i,j] for E in basis]
            rows.append(row)
    return matrix(R, 4, 9, rows)

B = boundary_matrix()
print("\nBoundary matrix B:")
show(B)



Boundary matrix B:


[ 0  0  0  0  1  0  0  0  0]
[ 0  1 -1  0  0  1  0  0  0]
[ 0  0  0  1  0  0 -1  1  0]
[ 0  0  0  0  1  0  0  0  0]

In [6]:
# Step 6: Flatten boundary of A
b_vec = vector(R, [B_A[i,j] for i in range(2) for j in range(2)])
print("\nFlattened boundary b_vec:")
show(b_vec)



Flattened boundary b_vec:


(x11, x01 - x02 + x12, x10 - x20 + x21, x11)

In [7]:
# Step 7: Solve B * v = b_vec for v

# Compute particular solution
sol = B.solve_right(b_vec)
print("\nParticular solution v0:")
show(sol)



Particular solution v0:


(0, x01 - x02 + x12, 0, x10 - x20 + x21, x11, 0, 0, 0, 0)

In [8]:
# Step 8: Compute kernel of B
ker = B.right_kernel()
print("\nKernel of B:")
show(ker.gens())



Kernel of B:


[
(1, 0, 0, 0, 0, 0, 0, 0, 0),
(0, 1, 1, 0, 0, 0, 0, 0, 0),
(0, 0, 1, 0, 0, 1, 0, 0, 0),
(0, 0, 0, 1, 0, 0, 1, 0, 0),
(0, 0, 0, 0, 0, 0, 1, 1, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 1)
]

In [9]:



# Step 9: Verify that flatten(A) - sol lies in kernel
v_A = vector(R, A.list())
diff = v_A - sol
print("\nDifference (A - v0):")
show(diff)

is_in_kernel = diff in ker
print("\nIs A in the solution space:", is_in_kernel)



Difference (A - v0):


(x00, x02 - x12, x02, x20 - x21, 0, x12, x20, x21, x22)


Is A in the solution space: True


In [10]:
import sys
sys.path.append("/home/sage/notebooks")

In [11]:
# Cell 1: Setup and imports
import sympy as sp
from simplicial_tensors.symbolic_tensor_ops import SymbolicTensor
from simplicial_tensors.tensor_ops import bdry, standard_basis_matrix

n = 3  # Shape (n, n) for 2D simplicial tensor

# Create symbolic tensor T
T = SymbolicTensor((n, n), init_type='range')

# Create a 1-horn and fill it
horn = T.horn(1)
T_filler = T.filler(horn, 1)

print("Original Tensor T:")
print(T.to_latex())

print("\nFiller Tensor T':")
print(T_filler.to_latex())


Original Tensor T:
\begin{bmatrix}
x_{0,0} & x_{0,1} & x_{0,2} \\
x_{1,0} & x_{1,1} & x_{1,2} \\
x_{2,0} & x_{2,1} & x_{2,2}
\end{bmatrix}

Filler Tensor T':
\begin{bmatrix}
x_{0,0} & x_{0,1} & x_{0,1} - x_{1,1} + x_{1,2} \\
x_{1,0} & x_{1,1} & x_{1,2} \\
x_{1,0} - x_{1,1} + x_{2,1} & x_{2,1} & x_{2,2}
\end{bmatrix}


In [12]:
# Cell 2: Compute difference and check cycle
T_diff = T.tensor - T_filler.tensor

print("\nDifference Tensor T - T':")
print(sp.pretty(T_diff))

# Check if cycle
T_diff_tensor = SymbolicTensor((n, n), tensor=T_diff)
T_diff_bdry = T_diff_tensor.bdry().tensor

print("\nBoundary of T - T':")
print(sp.pretty(T_diff_bdry))


is_cycle1 = all(c == 0 for c in T_diff_bdry[0]) 
is_cycle2 = all(c == 0 for c in T_diff_bdry[1])  # This works over Z
print("\nis_cycle1", is_cycle1)
print("\nis_cycle2", is_cycle2)
is_cycle = is_cycle1 and is_cycle2
print("\nIs T - T' a cycle:", is_cycle)




Difference Tensor T - T':
[[0 0 -x_{0,1} + x_{0,2} + x_{1,1} - x_{1,2}] 
                    [0 0 0]                   
 [-x_{1,0} + x_{1,1} + x_{2,0} - x_{2,1} 0 0]]

Boundary of T - T':
[[0 x_{0,1} - x_{0,2} - x_{1,1} + x_{1,2}] 
 [x_{1,0} - x_{1,1} - x_{2,0} + x_{2,1} 0]]

is_cycle1 False

is_cycle2 False

Is T - T' a cycle: False


In [13]:
n=3
print(n)

3


In [14]:

n=3
# Cell 3: Check if T - T' is a boundary (Linear Algebra)
from sage.all import *

# Polynomial ring for coefficients
R = PolynomialRing(ZZ, [f"x{i}{j}" for i in range(n) for j in range(n)])

# Build boundary matrix B
basis = [standard_basis_matrix(n, n, i, j) for i in range(n) for j in range(n)]
rows = []
for i in range(n - 1):
    for j in range(n - 1):
        row = [bdry(E)[i, j] for E in basis]
        rows.append(row)

B = matrix(ZZ, (n - 1) * (n - 1), n * n, rows)

# Flatten difference tensor
b_vec = vector(ZZ, [(T_diff[i, j] if isinstance(T_diff[i, j], int) else 0) for i in range(n - 1) for j in range(n - 1)])

# Solve B * v = b_vec
try:
    sol = B.solve_right(b_vec)
    print("\nT - T' is a boundary. Solution:")
    print(sol)
except ValueError:
    print("\nT - T' is not a boundary.")


TypeError: 'function' object cannot be interpreted as an integer

In [101]:
print(n)
print(range)
print(ZZ)
print(PolynomialRing)


<function numerical_approx at 0x7fa71bcab1a0>
<class 'range'>
Integer Ring
<function PolynomialRing at 0x7fa71994dee0>


In [102]:
del PolynomialRing
from sage.all import PolynomialRing
